# 7. Data Wrangling: Clean, Transform, Merge, Reshape
o 데이터 분석의 절차
 - 1) 문제의 정의 
 - 2) 데이터 준비: Extract(Data Gathering) - Transform (Cleaning, 변형, 정규화) - Loading 
 - 3) 데이터 탐구: 통계적 탐구, 
 - 4) 예측 모델링: 모델생성, 
 - 5) 분석/검증: 데이터마이닝 알고리즘 
 - 6) 결과 시각화: 시각화 및 해석
 - 7) 솔루션 보고 및 배포
 
o 데이터 분석 프로세스
 - 데이터 준비에 가장 많은 자원(시간, 비용, 인력)이 소모됨.
 
o pandas에서는 다양한 ETL 기능을 제공함.
 

In [71]:
from __future__ import division  # python 2에서 3에서 사용될 / 연산자의 사용법을 미리 사용 
from numpy.random import randn
import numpy as np
import os
# import matplotlib.pyplot as plt
np.random.seed(12345)
# plt.rc('figure', figsize=(10, 6))
from pandas import Series, DataFrame
import pandas
import pandas as pd
np.set_printoptions(precision=4, threshold=500)
pd.options.display.max_rows = 100

In [72]:
%matplotlib inline

ImportError: DLL load failed: 지정된 프로시저를 찾을 수 없습니다.

## 7.1 Combining and merging data sets
o pandas 객체에 저장된 데이터는 여러 내장 함수를 이용해 합치기 가능
 - pandas.merge: 하나 이상의 키를 기준으로 DataFrame의 로우를 합친다. SQL이나 다른 관계형 데이터 베이스의 join 연산과 유사
   (DataFrame.join 과 비교 예정) 
 - pandas.concat은 하나의 축을 따라 객체를 이어붙임
 - combine_first 인스턴스 메서드는 두 객체를 포개서 한 객체에서 누락된 데이터를 다른 객체에 있는 값으로 채울 수 있도록 한다.

### 7.1.1 Database-style DataFrame merges
o merge나 join 연산은 관계형 데이터베이스의 핵심적인 연산으로, 키를 하나 이상 사용해서 데이터 집합의 로우를 합침
 - merge key를 지정하지 않은 경우, 동일 컬럼명을 조인 키로 활용함. 

In [ ]:
df1 = DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                 'data1': range(7)})
df2 = DataFrame({'key': ['a', 'b', 'd'],
                 'data2': range(3)})
df1

In [ ]:
df2

In [ ]:
pd.merge(df1, df2)

In [ ]:
pd.merge(df1, df2, on='key')

In [ ]:
df3 = DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                 'data1': range(7)})
df4 = DataFrame({'rkey': ['a', 'b', 'd'],
                 'data2': range(3)})
pd.merge(df3, df4, left_on='lkey', right_on='rkey')

In [ ]:
pd.merge(df1, df2, how='outer')

In [ ]:
df1 = DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                 'data1': range(6)})
df2 = DataFrame({'key': ['a', 'b', 'a', 'b', 'd'],
                 'data2': range(5)})

In [ ]:
df1

In [ ]:
df2

In [ ]:
pd.merge(df1, df2, on='key', how='left')

In [ ]:
pd.merge(df1, df2, how='inner')

In [ ]:
left = DataFrame({'key1': ['foo', 'foo', 'bar'],
                  'key2': ['one', 'two', 'one'],
                  'lval': [1, 5, 4 ]})
right = DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],
                   'key2': ['one', 'one', 'one', 'two'],
                   'rval': [4, 5, 6, 7]})
# pd.merge(left, right, left_on='lval', right_on='rval', how='left')
pd.merge(left, right, on=['key1', 'key2'], how='outer')

In [ ]:
pd.merge(left,right)

In [29]:
pd.merge(left, right, on='key1')

NameError: name 'left' is not defined

In [ ]:
pd.merge(left, right, on='key1', suffixes=('_left', '_right'))

#### o merge 함수 인자 목록
 - left	머지하려는 DataFrame 중 왼쪽에 위치한 DataFrame
 - right	머지하려는 DataFrame 중 오른쪽에 위치한 DataFrame
 - how	조인방법. 'inner', 'outer', 'left', 'right'. 기본값은 inner
 - on	조인하려는 로우 이름. 반드시 두 DataFrame 객체 모두에 있는 이름이어야 한다. 만약 명시되지 않고 다른 조인 키도 주어지지 않으면 left와 right에서 공통되는 칼럼을 조인 키로 사용한다.
 - left_on	조인 키로 사용할 left DataFrame의 칼럼
 - right_on	조인 키로 사용할 right DataFrame의 칼럼
 - left_index	조인 키로 사용할 left DataFrame의 색인 로우(다중 색인일 경우의 키)
 - right_index	조인 키로 사용할 right DataFrame의 색인 로우(다중 색인일 경우의 키)
 - sort	조인 키에 따라 병합된 데이터를 사전 순으로 정렬. 기본값은 True. 대용량 데이터의 경우 False라면 성능상의 이득을 얻을 수도 있다.
 - suffixes	칼럼 이름이 겹칠 경우 각 칼럼 이름 뒤에 붙일 문자열의 튜플. 기본값은 ('_x', '_y'). 만약 'data'라는 칼럼 이름이 양쪽 DataFrame에 같이 존재하면 결과에서는 'data_x', 'data_y'로 나타난다.
 - copy	False일 경우 예외적인 경우에 결과로 데이터가 복사되지 않도록 한다. 기본값은 항상 복사가 이루어진다.

### 7.1.2 Merging on index

o 컬럼대신 index를 조인기준으로 활용하여 Merge 가능


In [ ]:
left1 = DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'],
                  'value': range(6)})
right1 = DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])

In [ ]:
left1

In [ ]:
right1

In [ ]:
pd.merge(left1, right1, left_on='key', right_index=True)

In [ ]:
pd.merge(left1, right1, left_on='key', right_index=True, how='outer')

In [ ]:
lefth = DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada'],
                   'key2': [2000, 2001, 2002, 2001, 2002],
                   'data': np.arange(5.)})
righth = DataFrame(np.arange(12).reshape((6, 2)),
                   index=[['Nevada', 'Nevada', 'Ohio', 'Ohio', 'Ohio', 'Ohio'],
                          [2001, 2000, 2000, 2000, 2001, 2002]],
                   columns=['event1', 'event2'])
lefth

In [ ]:
righth

In [ ]:
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True)

In [30]:
pd.merge(lefth, righth, left_on=['key1', 'key2'],
         right_index=True, how='outer')

NameError: name 'lefth' is not defined

In [ ]:
left2 = DataFrame([[1., 2.], [3., 4.], [5., 6.],[7.,8.]], index=['a', 'c', 'e','d'],
                 columns=['Ohio', 'Nevada'])
right2 = DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
                   index=['b', 'c', 'd', 'e'], columns=['Missouri', 'Alabama'])

In [ ]:
left2

In [ ]:
right2

In [ ]:
pd.merge(left2, right2, how='outer', left_index=True, right_index=True)

#### o DataFrame.join 
 - join 연산은 기본적으로 index를 기준으로 합병한다.
 - 기본적으로 호출하는 DataFrame을 outer join ( = Left Outer JOIN) 한다. 
 - 유사한(동일한) 색인구조를 갖을 경우 편리하게 사용 가능

In [ ]:
left2.join(right2)

In [ ]:
left2.join(right2, how='outer')

In [ ]:
left1.join(right1, on='key')

In [ ]:
another = DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]],
                    index=['a', 'c', 'e', 'f'], columns=['New York', 'Oregon'])
another

In [ ]:
left2

In [ ]:
right2

In [ ]:
left2.join([right2, another])

##### o outer : 3개 dataframe을 모두 보존  

In [ ]:
left2.join([right2, another], how='outer')

### 7.1.3 Concatenating along an axis
#### o 또 다른 합병의 종류: concatenating(이어붙이기), binding(연결), stacking(적층)
#### o concatenating
 - NumPy의 concatenate()

In [ ]:
arr = np.arange(12).reshape((3, 4))

In [ ]:
arr

In [ ]:
np.concatenate([arr, arr], axis=1)

In [ ]:
#s1 = Series([0, 1], index=['a', 'b'])
s1 = Series([0, 1], index=['a', 'c'])
s2 = Series([2, 3, 4], index=['c', 'd', 'e'])
s3 = Series([5, 6], index=['f', 'g'])

#### o Pandas.concat(Series, ...)
 - 디폴트 axis=0 을 기준으로 연결하여 붙임.(join 하지 않음. 단순 연결) 

In [ ]:
pd.concat([s1, s2, s3])

In [ ]:
pd.concat([s1, s2, s3], axis=1)

In [ ]:
s4 = pd.concat([s1 * 5, s3])
s4

In [ ]:
pd.concat([s1, s4], axis=1)

#### o pd.concat( )의 옵션
 - 참고) https://pandas.pydata.org/pandas-docs/stable/generated/pandas.concat.html
  - return: 
  - When concatenating all Series along the index (axis=0), a Series is returned. 
  - When objs contains at least one DataFrame, a DataFrame is returned. 
  - When concatenating along the columns (axis=1), a DataFrame is returned.

In [ ]:
s1

In [ ]:
s4

In [ ]:
pd.concat([s1, s4], axis=1)  # axis=1 일 경우에만,join 옵션(디폴트=outer)이 의미 있음. 
# pd.concat([s1, s4], axis=1, join='inner')

- join_axes 조인할 행의 축의 값을  지정

In [ ]:
pd.concat([s1, s4], axis=1, join_axes=[['a', 'c', 'b', 'e']])

- keys: Series를 이어붙일 경우, 다단계 색인을 통하여 소속 Series를 표기

In [ ]:
result = pd.concat([s1, s1, s3], keys=['one', 'two', 'three'])

In [ ]:
result

In [ ]:
# Much more on the unstack function later
result.unstack()

- keys : axis=1 이면, 
 - Series를 concat 한 결과는 DataFrame이 되고, 
 - keys 옵션은 결과 DataFrame의 컬럼이 된다.

In [ ]:
# pd.concat([s1, s2, s3])  # s1,s2,s3 확인 
pd.concat([s1, s2, s3], axis=1, keys=['one', 'two', 'three'])

#### o pd.concat(DataFrame+) 
 - Series와 동일하게 적용
 - keys: 기존 df에 컬럼명이 있으므로, 다단계 컬럼으로... 

In [ ]:
df1 = DataFrame(np.arange(6).reshape(3, 2), index=['a', 'b', 'c'],
                columns=['one', 'two'])
df2 = DataFrame(5 + np.arange(4).reshape(2, 2), index=['a', 'c'],
                columns=['three', 'four'])

In [ ]:
df1

In [ ]:
df2

In [ ]:
# pd.concat([df1, df2], axis=1, keys=['level1', 'level2'])
pd.concat([df1, df2], axis=1)
df3.pivot('one','two','three')

In [ ]:
df3=pd.concat([df1, df2], axis=1)
# df1.unstack()

In [ ]:
pd.concat({'level1': df1, 'level2': df2}, axis=1)

In [ ]:
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'],
          names=['upper', 'lower'])

In [ ]:
df1 = DataFrame(np.random.randn(3, 4), columns=['a', 'b', 'c', 'd'])
df2 = DataFrame(np.random.randn(2, 3), columns=['b', 'd', 'a'])

In [ ]:
df1

In [ ]:
df2

- ignore_index = TRUE : index가 불필요한 경우
  - 참고) 디폴트 값: axix=0 

In [ ]:
pd.concat([df1, df2], ignore_index=True)

### 7.1.4 겹치는 데이터 합치기

In [ ]:
a = Series([np.nan, 2.5, np.nan, 3.5, 4.5, np.nan],
           index=['f', 'e', 'd', 'c', 'b', 'a'])
b = Series(np.arange(len(a), dtype=np.float64),
           index=['f', 'e', 'd', 'c', 'b', 'a'])
b[-1] = np.nan

In [ ]:
a

In [ ]:
b

#### o np.where(condition[, x, y] ) 
 - 참고: https://docs.scipy.org/doc/numpy-1.14.0/reference/generated/numpy.where.html
 - condition, x, y 는 array-like 객체
 - condition이 TRUE이면 x, 아니면 y 선택
 - return: ndarray or tuple of ndarrays

In [ ]:
np.where(pd.isnull(a), b, a) # a가 null 인 곳만, b가 리턴됨 

#### Series.combine_first( ) 
 - 참고) https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.combine_first.html
 - 호출 Series의 값을 우선하고 NULL인 경우나 callee에만 있는 값은 calㅣee의 값으로 넣는다. 
 - 색인값으로 정렬 

In [ ]:
b[:-2]

In [ ]:
a[2:]

In [ ]:
b[:-2].combine_first(a[2:])

In [ ]:
df1 = DataFrame({'a': [1., np.nan, 5., np.nan],
                 'b': [np.nan, 2., np.nan, 6.],
                 'c': range(2, 18, 4)})
df2 = DataFrame({'a': [5., 4., np.nan, 3., 7.],
                 'b': [np.nan, 3., 4., 6., 8.]})
df1.combine_first(df2)

## 7.2 재형성과 피봇(Reshaping and pivoting)

### 7.2.1 Reshaping with hierarchical indexing
#### o stack(level=-1, dropna=True) 
 - 데이터 컬럼을 가장 안쪽의 색인으로 피봇 또는 회전시킨다. 
 - dropna=TRUE 이므로, 누락된 값은 자동으로 사라짐.(참고: 아래)
 - 참고) https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.stack.html

#### o unstack(level=-1, fill_value=None): 
 - 로우를 컬럼의 가장안쪽 레벨로 피봇 또는 회전시킨다. 
 -  fill_value=None 이므로, 불일치하는 피봇 색인들은 Nan으로 채워짐.(참고: 아래)
 - 참고) https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.unstack.html


In [ ]:
data = DataFrame(np.arange(6).reshape((2, 3)),
                 index=pd.Index(['Ohio', 'Colorado'], name='state'),
                 columns=pd.Index(['one', 'two', 'three'], name='number'))
data

In [ ]:
result = data.stack()
result

In [ ]:
result.unstack()

In [ ]:
result.unstack(0)

In [ ]:
result.unstack('state')

In [ ]:
s1 = Series([0, 1, 2, 3], index=['a', 'b', 'c', 'd'])
s2 = Series([4, 5, 6], index=['c', 'd', 'e'])
data2 = pd.concat([s1, s2], keys=['one', 'two']) # keys가 원래의 Series을 알 수 있도록 색인레벨 부여
data2

- unstack()의 경우, povot 레벨의 색인이 일치하지 않으면, Nan값 발생

In [ ]:
data2.unstack()

In [ ]:
data2.unstack().stack()

In [ ]:
data2.unstack().stack(dropna=False)

- unstack() : 이동 목표위치는 가장 안쪽 컬럼레벨이다.

In [ ]:
df = DataFrame({'left': result, 'right': result + 5},
               columns=pd.Index(['left', 'right'], name='side'))
df

In [ ]:
df.unstack('state')

In [ ]:
df.unstack('state').stack('side')

### 7.2.2 피봇팅으로 데이터 나열방식 바꾸기 (Pivoting "long" to "wide" format)

In [ ]:
data = pd.read_csv('ch07/macrodata.csv')
data

In [ ]:
periods = pd.PeriodIndex(year=data.year, quarter=data.quarter, name='date')
periods

#### DataFrame.to_records(index=True, convert_datetime64=True)
 - Convert DataFrame to record array. 
 - 참고) https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_records.html

#### PeriodIndex.to_timestamp(freq=None, how='start')
 - Cast to DatetimeIndex
 - 참고) https://pandas.pydata.org/pandas-docs/stable/generated/pandas.PeriodIndex.to_timestamp.html
   - freq : string or DateOffset, default ‘D’ for week or longer, ‘S’
otherwise. 
   - how : {‘s’, ‘e’, ‘start’, ‘end’}

In [ ]:
data = DataFrame(data.to_records(),
                 columns=pd.Index(['realgdp', 'infl', 'unemp'], name='item'),
                 index=periods.to_timestamp('D', 'end'))
data[:10]

In [ ]:
# ldata = data.stack().reset_index()#.rename(columns={0: 'value'})
# ldata
ldata = data.stack().reset_index().rename(columns={0: 'value'})
ldata[:10]

#### DataFrame.pivot(index=None, columns=None, values=None)
 - dataframe을 지정된 각 값들을 index, column, values로 사용하도록 피봇팅한 결과 산출 
 - 참고) https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.pivot.html

In [ ]:
wdata = ldata.pivot('date', 'item', 'value')
wdata[:10]

In [ ]:
ldata[:10]

In [ ]:
pivoted = ldata.pivot('date', 'item', 'value')
pivoted.head()

In [ ]:
ldata['value2'] = np.random.randn(len(ldata))
ldata[:10]

- values=None이면, 나머지 모든 컬럼이 "values="의 값으로 지정됨. 

In [ ]:
pivoted = ldata.pivot('date', 'item')
pivoted[:5]

In [ ]:
pivoted['value'][:5]

In [ ]:
#unstacked = ldata.set_index(['date', 'item'])#.unstack('item')
#unstacked[:7]
unstacked = ldata.set_index(['date', 'item']).unstack('item')
unstacked[:7]

## 7.3 데이터 변형(Data transformation)

### 7.3.1 중복제거(Removing duplicates)

In [ ]:
data = DataFrame({'k1': ['one'] * 3 + ['two'] * 4,
                  'k2': [1, 1, 2, 3, 3, 4, 4]})
data

#### dataframe.duplicated(subset=None, keep='first')
 - Return boolean Series denoting duplicate rows, optionally only considering certain columns
 - keep 값(first, last, False) 에 따라서, False의 위치를 결정 
   - 참고: keep이므로 duplicate하지 않은 것(keep할 것)으로 판정(false)할 것을 지정, 
   - 따라서, False는 keep할 것이 없으므로, 중볻된 모든 항목이 TRUE
   

In [ ]:
data.duplicated()

In [ ]:
data.drop_duplicates()

In [ ]:
data['v1'] = range(7)
data

In [ ]:
data.drop_duplicates(['k1'])

In [31]:
data.drop_duplicates(['k1', 'k2'], keep='last')

NameError: name 'data' is not defined

### 7.3.2 함수나 매핑으로 데이터 변형하기(Transforming data using a function or mapping)
- 필요성: 값에 따라 데이터의 모양 변경 필요

In [ ]:
data = DataFrame({'food': ['bacon', 'pulled pork', 'bacon', 'Pastrami',
                           'corned beef', 'Bacon', 'pastrami', 'honey ham',
                           'nova lox'],
                  'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

In [ ]:
meat_to_animal = {
  'bacon': 'pig',
  'pulled pork': 'pig',
  'pastrami': 'cow',
  'corned beef': 'cow',
  'honey ham': 'pig',
  'nova lox': 'salmon'
}

#### o Series.map(arg, na_action=None)
 - Map values of Series using input correspondence (which can be a dict, Series, or function)
  - dict: key -> value, 
  - Series: index -> value
  - Function: input -> funtion return 

In [ ]:
data['animal'] = data['food'].map(str.lower).map(meat_to_animal)
data

In [ ]:
data['food'].map(lambda x: meat_to_animal[x.lower()])

### 7.3.3 값 치환하기(Replacing values)
#### o Series.replace(to_replace=None, value=None, inplace=False, limit=None, regex=False, method='pad', axis=None)
 - https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.replace.html
 - to_replace : str, regex, list, dict, Series, numeric, or None
 - value : scalar, dict, list, str, regex, default None 

In [ ]:
data = Series([1., -999., 2., -999., -1000., 3.])
data

In [ ]:
data.replace(-999, np.nan)

In [ ]:
data.replace([-999, -1000], np.nan)

In [32]:
data.replace([-999, -1000], [np.nan, 0])

NameError: name 'data' is not defined

In [33]:
data.replace({-999: np.nan, -1000: 0})

NameError: name 'data' is not defined

### 7.3.4 DataFrame 축 색인 이름변경(Renaming axis indexes)
##### o Index.map(mapper)
 - Series와 유사하게, Index에 map 함수 보유 
 - Apply mapper function to an index
   - mapper : callable

In [ ]:
data = DataFrame(np.arange(12).reshape((3, 4)),
                 index=['Ohio', 'Colorado', 'New York'],
                 columns=['one', 'two', 'three', 'four'])
data

In [ ]:
data.index.map(str.upper)

In [ ]:
data.index = data.index.map(str.upper)
data

#### o DataFrame.rename(mapper=None, index=None, columns=None, axis=None, copy=True, inplace=False, level=None)
 - https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.rename.html
 - Alter axes labels.
 - mapper, index, columns : dict-like or function, optional
  - Function / dict values must be unique (1-to-1). 
  - Labels not contained in a dict / Series will be left as-is.
  - Extra labels listed don’t throw an error.
 - copy : boolean, default True  ==> rename copies and return new DataFrame

In [ ]:
# (mapper, index, columns)값이 함수인경우 
data.rename(index=str.title, columns=str.upper)

In [ ]:
# (mapper, index, columns)값이 사전인경우 
data.rename(index={'OHIO': 'INDIANA'},
            columns={'three': 'peekaboo'})

In [ ]:
# Always returns a reference to a DataFrame
_ = data.rename(index={'OHIO': 'INDIANA'}, inplace=True)
data  # Look at inplcace=TRUE

### 7.3.5 개별화와 양자화(Discretization and binning)
- 필요성: 데이터를 분할하거나 그룹화에 대한 필요는 상존

#### o class pandas.Categorical(values, categories=None, ordered=None, dtype=None, fastpath=False)
- https://pandas.pydata.org/pandas-docs/stable/categorical.html
- Represents a categorical variable in classic R / S-plus fashion
- Categoricals can only take on only a limited, and usually fixed, number of possible values (categories)
- All values of the Categorical are either in categories or np.nan.
- Order is defined by the order of the categories, not lexical order of the values
- Parameters:
  - values : list-like
    - The values of the categorical. If categories are given, values not in categories will be replaced with NaN.
  - categories : Index-like (unique), optional
    - The unique categories for this categorical. If not given, the categories are assumed to be the unique values of values.
  - ordered : boolean, (default False)
- 결론적으로, 
  - 1) categories가 주어지면, valeuse는 분류될 값이고, categories는 분류기준을 제공
  - 2) categories가 안 주어지면, valeuse는 분류될 값이자, categories 자체가 됨. 
- Categorical.codes
  - Level codes are an array if integer which are the positions of the real values in the categories array.  - 

In [ ]:
# Example 1 of Categorical
pd.Categorical([1, 2, 3, 1, 2, 3])

In [ ]:
# Example 2 of Categorical
pd.Categorical(['a', 'b', 'c', 'a', 'b', 'c'])

In [ ]:
c = pd.Categorical(['a','b','c','a','b','c'], ordered=True, categories=['c', 'b', 'a'])
c

#### o pandas.cut(x, bins, right=True, labels=None, retbins=False, precision=3, include_lowest=False)
- https://pandas.pydata.org/pandas-docs/stable/generated/pandas.cut.html
- Return indices of half-open bins to which each value of x belongs.
 - out: Categorical or Series or array of integers
 - bins:ndarray of floats only if retbins is True.
- Parameters
 - x : array-like
   - Input array to be binned. It has to be 1-dimensional.
 - bins : int, sequence of scalars, or IntervalIndex
   - If bins is an int, it defines the number of equal-width bins in the range of x. 
     - However, in this case, the range of x is extended by .1% on each side to include the min or max values of x
   - If bins is a sequence it defines the bin edges allowing for non-uniform bin width
 - right : bool, optional
   - Indicates whether the bins include the rightmost edge or not.
 - labels : array or boolean, default None
   - Used as labels for the resulting bins. Must be of the same length as the resulting bins. 
   - If False, return only integer indicators of the bins.

In [ ]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

In [ ]:
bins = [18, 25, 35, 60, 100]
cats = pd.cut(ages, bins)
cats

In [ ]:
cats.codes

In [ ]:
pd.value_counts(cats)

In [ ]:
# right : 경계포함(대괄호)의 위치를 결정
pd.cut(ages, [18, 26, 36, 61, 100], right=False)

In [ ]:
# lables 
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']
pd.cut(ages, bins, labels=group_names)

In [ ]:
data = np.random.rand(20)
data

In [ ]:
cut = pd.cut(data, 4)
cut

In [ ]:
cut.value_counts()

In [ ]:
mydata=np.arange(10)
mydata

In [ ]:
# bins 가 integer일 경우, 1% of range가 min, max에서 확대될 수 있다. see min boundary
mycut = pd.cut(mydata, 4)
mycut

#### o pandas.qcut(x, q, labels=None, retbins=False, precision=3, duplicates='raise')
- Quantile-based discretization function. 
- Discretize variable into equal-sized buckets based on rank or based on sample quantiles. 
  - quantile: 변위치, 예) 4분위, 10%분위 등
- For example 1000 values for 10 quantiles would produce a Categorical object indicating quantile membership for each data point.
- Parameters:
  - q : integer or array of quantiles, Number of quantiles. Alternately array of quantiles, e.g. [0, .25, .5, .75, 1.]
- cut() vs. qcut()
 - cut(): 구간길기가(bin의 길이) 값을 기준으로 동일
 - qcut(): 변위치를 사용하기 때문에, value_counts()가 동일하도록 구간길기를 지정 (각 bin에 소속되는 갯수가 동일)

In [ ]:
data = np.random.randn(1000) # Normally distributed
cats = pd.qcut(data, 4) # Cut into quartiles
cats

In [ ]:
pd.value_counts(cats)

In [ ]:
# 변위치를 직접 지정하는 경우 
cats=pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.])
cats

In [ ]:
pd.value_counts(cats)

### 7.3.6 특이값 찾아내고 제외하기 (Detecting and filtering outliers)

In [ ]:
np.random.seed(12345)
data = DataFrame(np.random.randn(1000, 4))
data.describe()

In [ ]:
col = data[3]

In [ ]:
col[np.abs(col) > 3]

In [ ]:
# 3이 넘어가는 값이 1개 이상 존재하는 행을 선택 
data[(np.abs(data) > 3).any(1)]

In [ ]:
data[np.abs(data) > 3] = np.sign(data) * 3 # numpy.sign()  사인함수 값
data.describe()

### 7.3.7 Permutation and random sampling
#### o numpy.random.permutation(x)
- Randomly permute a sequence, or return a permuted range.
- If x is a multi-dimensional array, it is only shuffled along its first index.
- parameters
 - x : int or array_like
   - If x is an integer, randomly permute np.arange(x).
   - f x is an array, make a copy and shuffle the elements randomly.
- returns 
 - out : ndarray 
   - Permuted sequence or array range.

In [ ]:
df = DataFrame(np.arange(5 * 4).reshape((5, 4)))
sampler = np.random.permutation(5)
sampler

In [ ]:
df

#### o DataFrame.take(indices, axis=0, convert=None, is_copy=True, **kwargs)
- https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.take.html
- Return the elements in the given positional indices along an axis
- Parameters:
 - indices : array-like
 - axis : int, default 0

In [ ]:
df.take(sampler)

- 일부만 선택하고자 하는 경우

In [ ]:
df.take(np.random.permutation(len(df))[:3])

- 주어진 값으로만 치환을 통해 샘플을 생성하는 방법
 - 예) 주어진 bag의 값을, 10개의 샘플을 생성 

In [ ]:
bag = np.array([5, 7, -1, 6, 4])
sampler = np.random.randint(0, len(bag), size=10)

In [ ]:
sampler

In [ ]:
# numpy.take(a, indices, axis=None, out=None, mode='raise')  - df.take()와 거의 유사한 인터페이스
draws = bag.take(sampler)
draws

### 7.3.8 표시자/더미변수 (Computing indicator / dummy variables)
#### o 용도: 값의 종류에 따라 컬럼을 만들고, 그 값의 존재에 따라 1/0을 설정
#### o pandas.get_dummies(data, prefix=None, prefix_sep='_', dummy_na=False, columns=None, sparse=False, drop_first=False)
- https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html
- Convert categorical variable into dummy/indicator variables
 - data : array-like, Series, or DataFrame
 - prefix : string, list of strings, or dict of strings, default None
   - String to append DataFrame column names

In [ ]:
df = DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                'data1': range(6)})
pd.get_dummies(df['key'])

In [ ]:
dummies = pd.get_dummies(df['key'], prefix='key')
dummies

In [ ]:
df_with_dummy = df[['data1']].join(dummies)
df_with_dummy

In [ ]:
mnames = ['movie_id', 'title', 'genres']
movies = pd.read_table('ch02/movielens/movies.dat', sep='::', header=None,names=mnames,engine='python')
movies[:10]

In [ ]:
#mytitle=movies.title.replace(')','')
#mytitle[:10]
title_iter = {}
set.add((x.split('(')[-1]) for x in movies.title[:3]))
titles = sorted(set.unititle))[-1]
tite[-1s

#### o str.split(str="", num=string.count(str))
- https://www.tutorialspoint.com/python/string_split.htm
- returns a list of all the words in the string, using parameter str as the separator

In [ ]:
genre_iter = (set(x.split('|')) for x in movies.genres)
genres = sorted(set.union(*genre_iter))
genres

In [ ]:
dummies = DataFrame(np.zeros((len(movies), len(genres))), columns=genres)
dummies

In [ ]:
#확인: movies.genres
# movies.genres[1].split('|')
for i, gen in enumerate(movies.genres):
    dummies.ix[i, gen.split('|')] = 1
# 결과 확인
dummies
# dummies.add_prefix('Genre_')

In [ ]:
movies_windic = movies.join(dummies.add_prefix('Genre_'))
movies_windic.ix[0]

#### o 통계에 활용: pd.cut()과 get_dummies() 이용한 통계 데이터 활용

In [ ]:
np.random.seed(12345)

In [ ]:
values = np.random.rand(10)
values

In [ ]:
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]
pd.get_dummies(pd.cut(values, bins))

## 7.4 String manipulation
#### o 텍스트 연산의 대부분은 문자열 객체의 내장함수를 이용해 처리 가능
#### o 보다 복잡한 패턴매칭은 정규식(regular expression)활용 - pandas는 배열에 RE 적용 가능

### 7.4.1 String object methods
- https://docs.python.org/2.7/library/string.html#string-functions
- https://www.programiz.com/python-programming/methods/string

In [ ]:
val = 'a,b,  guido'
val.split(',')

In [ ]:
pieces = [x.strip() for x in val.split(',')]
pieces

In [ ]:
first, second, third = pieces
first + '::' + second + '::' + third

In [ ]:
'::'.join(pieces)

In [ ]:
'guido' in val

In [ ]:
val.index(',')

In [ ]:
#string.find(): 없을 경우 -1 return
val.find(':')

In [ ]:
#string.index(): 없을 경우 에러 발생
val.index(':')

In [ ]:
val.count(',')

In [ ]:
val.replace(',', '::')

In [ ]:
val.replace(',', '')

### 7.4.2 Regular expressions
#### o 파이썬에서는 re 모듈이 내장
- 학습: 
 - http://highcode.tistory.com/6
 - http://www.nextree.co.kr/p4327/
- 참고
 - "RegEx Learn the Hard way"는 유료화 
 - https://docs.python.org/3/library/re.html 

In [34]:
import re
text = "foo    bar\t baz  \tqux"
re.split('\s+', text)

['foo', 'bar', 'baz', 'qux']

- re.compile() 결과를 활용하는 것이, computing power 소모 감소 

In [ ]:
regex = re.compile('\s+')
regex.split(text)

#### o Regular Expression Objects
- https://docs.python.org/3/library/re.html#regex-objects   - 6.2.3. Regular Expression Objects
- regex.findall(string[, pos[, endpos]])
  - 매치되는 모든 문자열을 찾아 줌.
  - Similar to the findall() function, using the compiled pattern, but also accepts optional pos and endpos parameters that limit the search region like for search()  
- regex.search(string[, pos[, endpos]])
 - 매치되는 첫번째 문자열을 찾아줌
 - pos: 매칭을 시작하는 문자열위치(0을 시작기준으로), endpoos: 매칭종료하는 문자열위치
- regex.match(string[, pos[, endpos]])
 - 시작되는 부분에서 매치되는 첫번째 문자열을 찾아줌

In [ ]:
regex.findall(text)

In [69]:
text = """Dave dave@google.com
Steve lala+@google.com
Rob hello@kpu.ac.kr
Ryan ryan_james@yahoo.com
Nadal nadal%tennis@yahoo.com
Federa federa@main.tennis.yahoo.spain.sp
"""
pattern= '[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{3,4}'

# re.IGNORECASE makes the regex case-insensitive
regex = re.compile(pattern, flags=re.IGNORECASE)

In [70]:
regex.findall(text)

['dave@google.com',
 'lala+@google.com',
 'ryan_james@yahoo.com',
 'nadal%tennis@yahoo.com',
 'federa@main.tennis.yahoo.spai']

In [ ]:
m = regex.search(text)
m

In [ ]:
text[m.start():m.end()]

In [44]:
regex.match(text)
print(regex.match(text))

None


In [ ]:
print(regex.sub('REDACTED', text))

In [39]:
pattern = r'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})'
regex = re.compile(pattern, flags=re.IGNORECASE)

In [40]:
# 위 URL (https://docs.python.org/3/library/re.html#regex-objects )의 
# 6.2.4. Match Objects 참고 
m = regex.match('wesm@bright.net')
m.groups()

('wesm', 'bright', 'net')

In [41]:
regex.findall(text)

[('dave', 'google', 'com'),
 ('steve', 'gmail', 'com'),
 ('rob', 'gmail', 'com'),
 ('ryan', 'yahoo', 'com')]

In [42]:
# \1 \2 \3 print문과 같이 text를 활용:
print(regex.sub(r'Username: \1, Domain: \2, Suffix: \3', text))

Dave Username: dave, Domain: google, Suffix: com
Steve Username: steve, Domain: gmail, Suffix: com
Rob Username: rob, Domain: gmail, Suffix: com
Ryan Username: ryan, Domain: yahoo, Suffix: com



In [26]:
regex = re.compile(r"""
    (?P<username>[A-Z0-9._%+-]+)
    @
    (?P<domain>[A-Z0-9.-]+)
    \.
    (?P<suffix>[A-Z]{2,4})""", flags=re.IGNORECASE|re.VERBOSE)

NameError: name 're' is not defined

In [ ]:
m = regex.match('wesm@bright.net')
m.groupdict()

### 7.4.3 Vectorized string functions in pandas

In [34]:
data = {'Dave': 'dave@google.com', 'Steve': 'steve@gmail.com',
        'Rob': 'rob@gmail.com', 'Wes': np.nan}
data = Series(data)

In [35]:
data

Dave     dave@google.com
Rob        rob@gmail.com
Steve    steve@gmail.com
Wes                  NaN
dtype: object

In [36]:
data.isnull()

Dave     False
Rob      False
Steve    False
Wes       True
dtype: bool

In [37]:
data.str.contains('gmail')

Dave     False
Rob       True
Steve     True
Wes        NaN
dtype: object

In [39]:
pattern

NameError: name 'pattern' is not defined

In [584]:
data.str.findall(pattern, flags=re.IGNORECASE)

Dave     [(dave, google, com)]
Rob        [(rob, gmail, com)]
Steve    [(steve, gmail, com)]
Wes                        NaN
dtype: object

In [587]:
matches = data.str.match(pattern, flags=re.IGNORECASE)
matches

Dave     True
Rob      True
Steve    True
Wes       NaN
dtype: object

In [588]:
matches.str.get(1)

Dave    NaN
Rob     NaN
Steve   NaN
Wes     NaN
dtype: float64

In [589]:
matches.str[0]

Dave    NaN
Rob     NaN
Steve   NaN
Wes     NaN
dtype: float64

In [591]:
data.str[:5]

Dave     dave@
Rob      rob@g
Steve    steve
Wes        NaN
dtype: object

## 7.5 HomeWork: Example: USDA Food Database
#### o 교재의 내용실행과 더불어 2개 이상의 다른 분석구조를 갖는 분석값을 제시하시오.
- 제출형태: jupyter notebook
- 포함내용: 분석내용에 대한 설명
- Due: 5/15(화) 자정

In [16]:
import json
db = json.load(open('ch07/foods-2011-10-03.json'))
len(db)

6636

In [17]:
db[0].keys()

[u'portions',
 u'description',
 u'tags',
 u'nutrients',
 u'group',
 u'id',
 u'manufacturer']

In [18]:
db[0]['nutrients'][0]

{u'description': u'Protein',
 u'group': u'Composition',
 u'units': u'g',
 u'value': 25.18}

In [19]:
nutrients = DataFrame(db[0]['nutrients'])
nutrients[:7]

,description,group,units,value
0,Protein,Composition,g,25.18
1,Total lipid (fat),Composition,g,29.20
2,"Carbohydrate, by difference",Composition,g,3.06
3,Ash,Other,g,3.28
4,Energy,Energy,kcal,376.00
5,Water,Composition,g,39.28
6,Energy,Energy,kJ,1573.00


In [20]:
info_keys = ['description', 'group', 'id', 'manufacturer']
info = DataFrame(db, columns=info_keys)

In [30]:
info[:5]

,description,group,id,manufacturer
0,"Cheese, caraway",Dairy and Egg Products,1008,
1,"Cheese, cheddar",Dairy and Egg Products,1009,
2,"Cheese, edam",Dairy and Egg Products,1018,
3,"Cheese, feta",Dairy and Egg Products,1019,
4,"Cheese, mozzarella, part skim milk",Dairy and Egg Products,1028,


In [31]:
info

,description,group,id,manufacturer
0,"Cheese, caraway",Dairy and Egg Products,1008,
1,"Cheese, cheddar",Dairy and Egg Products,1009,
2,"Cheese, edam",Dairy and Egg Products,1018,
3,"Cheese, feta",Dairy and Egg Products,1019,
4,"Cheese, mozzarella, part skim milk",Dairy and Egg Products,1028,
5,"Cheese, mozzarella, part skim milk, low moisture",Dairy and Egg Products,1029,
6,"Cheese, romano",Dairy and Egg Products,1038,
7,"Cheese, roquefort",Dairy and Egg Products,1039,
8,"Cheese spread, pasteurized process, american, ...",Dairy and Egg Products,1048,
9,"Cream, fluid, half and half",Dairy and Egg Products,1049,


In [32]:
pd.value_counts(info.group)[:10]

Vegetables and Vegetable Products    812
Beef Products                        618
Baked Products                       496
Breakfast Cereals                    403
Legumes and Legume Products          365
Fast Foods                           365
Lamb, Veal, and Game Products        345
Sweets                               341
Fruits and Fruit Juices              328
Pork Products                        328
Name: group, dtype: int64

In [33]:
nutrients = []

for rec in db:
    fnuts = DataFrame(rec['nutrients'])
    fnuts['id'] = rec['id']
    nutrients.append(fnuts)

nutrients = pd.concat(nutrients, ignore_index=True)

In [27]:
nutrients

,description,group,units,value,id
0,Protein,Composition,g,25.180,1008
1,Total lipid (fat),Composition,g,29.200,1008
2,"Carbohydrate, by difference",Composition,g,3.060,1008
3,Ash,Other,g,3.280,1008
4,Energy,Energy,kcal,376.000,1008
5,Water,Composition,g,39.280,1008
6,Energy,Energy,kJ,1573.000,1008
7,"Fiber, total dietary",Composition,g,0.000,1008
8,"Calcium, Ca",Elements,mg,673.000,1008
9,"Iron, Fe",Elements,mg,0.640,1008


In [28]:
nutrients.duplicated().sum()

14179

In [29]:
nutrients = nutrients.drop_duplicates()

In [ ]:
col_mapping = {'description' : 'food',
               'group'       : 'fgroup'}
info = info.rename(columns=col_mapping, copy=False)
info

In [ ]:
col_mapping = {'description' : 'nutrient',
               'group' : 'nutgroup'}
nutrients = nutrients.rename(columns=col_mapping, copy=False)
nutrients

In [ ]:
ndata = pd.merge(nutrients, info, on='id', how='outer')

In [ ]:
ndata

In [ ]:
ndata.ix[30000]

In [ ]:
result = ndata.groupby(['nutrient', 'fgroup'])['value'].quantile(0.5)
result['Zinc, Zn'].order().plot(kind='barh')

In [ ]:
by_nutrient = ndata.groupby(['nutgroup', 'nutrient'])

get_maximum = lambda x: x.xs(x.value.idxmax())
get_minimum = lambda x: x.xs(x.value.idxmin())

max_foods = by_nutrient.apply(get_maximum)[['value', 'food']]

# make the food a little smaller
max_foods.food = max_foods.food.str[:50]

In [ ]:
max_foods.ix['Amino Acids']['food']